#### LangChain Essentials Course

# LangChain Agents Intro

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Agents, adding the ability to use tools such as search and calculators to complete tasks that normal LLMs cannot fufil. We'll provide an example for both OpenAI's `gpt-4o-mini`.

## Introduction to Tools

Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the `@tool` decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

* A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.

* Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.

* Both parameter and return type annotations.

In [2]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [3]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000001BA4081DBC0>)

We can see the tool name, description, and arg schema:

In [4]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [5]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [7]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [8]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [9]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

Now we also want this agent to remember previous iterations of the conversations, to do that, we need to add a placeholder to pass into the prompt which will then get passed into the LLM as our memory.

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

We start by initializing the 1B parameter Llama 3.2 model, fine-tuned for instruction following. We pull the model from Ollama by switching to our terminal and executing:

ollama pull llama3.2:1b-instruct-fp16

Once the model has finished downloading, we initialize it in LangChain using the ChatOllama class:

In [ ]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b-instruct-fp16"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

When creating an agent we need to create memory to make the agent remember previous iterations of the chat history.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

Now we need to settup the agent to handle our tools, to do this, we use the create_tool_calling_agent function, and then using a executor agent we can then call that agent when we need to do so.

In [43]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

tools = [add, subtract, multiply, exponentiate]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

Here we can test the output's of our agent, firstly we tell it our name, then we will try and get it to call some tools we gave it earlier, then we will try and get the agent to recall our name.

In [44]:
agent_executor.invoke({"input": "My name is Josh", "chat_history": memory})



> Entering new AgentExecutor chain...
Nice to meet you, Josh! How can I assist you today?

> Finished chain.


{'input': 'My name is Josh',
 'chat_history': [HumanMessage(content='My name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Josh! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, Josh! How can I assist you today?'}

In [45]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3", 
    "chat_history": memory})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 9, 'y': 10}`


19.0
Invoking: `multiply` with `{'x': 4, 'y': 2}`


8.0
Invoking: `exponentiate` with `{'x': 2, 'y': 3}`


8.0
Invoking: `subtract` with `{'x': 19, 'y': 8}`


-11.0The result of the expression \(9 + 10 - 4 \times 2^3\) is \(-11\).

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='My name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Josh! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of the expression \\(9 + 10 - 4 \\times 2^3\\) is \\(-11\\).', additional_kwargs={}, response_metadata={})],
 'output': 'The result of the expression \\(9 + 10 - 4 \\times 2^3\\) is \\(-11\\).'}

In [46]:
agent_executor.invoke({"input": "What is my name", "chat_history": memory})



> Entering new AgentExecutor chain...
Your name is Josh.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='My name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Josh! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of the expression \\(9 + 10 - 4 \\times 2^3\\) is \\(-11\\).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Josh.', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Josh.'}

## SerpAPI Weather Example

Let's use [Serpapi](https://serpapi.com/users/sign_in) from the load_tools function, this will allow us to search the current web. To do this too, you will need a SERPAPI_API_KEY, you can get a free plan which will grant you 100 searches per week.

In [47]:
from langchain.agents import load_tools
import datetime

import os
os.environ["SERPAPI_API_KEY"] = ""

Here we will pull a tool from the load_tools function LangChain offers, this will pull our SerpAPI which allows us to search the web for answers, however this does require an API key as mentioned above.

In [48]:
toolbox = load_tools(
   ['serpapi'],
    llm
)

These custom tools can look into your IP address, find out where you are currently, then we will also use a secondary function to get the current date and time, then we will use this information to feed into the SerpAPI to find us the weather pattern in your area and at the time of the function calling.

In [49]:
import requests

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get('https://ipinfo.io/json')
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            return  f"""Latitude: {latitude}, 
                    Longitude: {longitude}, 
                    City: {data.get('city', 'N/A')}, 
                    Country: {data.get('country', 'N/A')}"""
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"
    
@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

Now because we don't want a memory variable involved with this AI we can get rid of the chat_history variable which was being used in the prompt previously. 

In [50]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

This is all mainly the same as before, but without the memory and a new set of tools for us to use.

In [51]:
tools = [toolbox[0], get_current_datetime, get_location_from_ip]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

For me I have to specify to the AI as I live in the UK and alot of places in the UK also exist in USA, which is why I explicitly state to use the country in the search as well as the town / city.

In [52]:
agent_executor.invoke({"input": """firstly what is the date and time right now, 
                       then what is the weather like is where I am in right now but 
                       becareful as there can be more then two places on earth with 
                       the same name?"""})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`


2024-11-15 10:26:53
Invoking: `get_location_from_ip` with `{}`


Latitude: 53.1333, Longitude: -1.2000, City: Mansfield, Country: GB
Invoking: `Search` with `Mansfield weather`


{'type': 'weather_result', 'temperature': '48', 'unit': 'Fahrenheit', 'precipitation': '6%', 'humidity': '97%', 'wind': '7 mph', 'location': 'Mansfield, OH', 'date': 'Friday 5:00 AM', 'weather': 'Cloudy'}The current date and time is **November 15, 2024, 10:26 AM**. 

You are located in **Mansfield, UK**. The weather there is **cloudy**, with a temperature of **48°F**, humidity at **97%**, and wind speed of **7 mph**. There is a **6% chance of precipitation**.

> Finished chain.


{'input': 'firstly what is the date and time right now, then what is the weather like is where I am in right now but becareful as there can be more then two places on earth with the same name?',
 'output': 'The current date and time is **November 15, 2024, 10:26 AM**. \n\nYou are located in **Mansfield, UK**. The weather there is **cloudy**, with a temperature of **48°F**, humidity at **97%**, and wind speed of **7 mph**. There is a **6% chance of precipitation**.'}